## Deployment on production and future

In [ ]:
!pip install -q fastapi uvicorn langchain langchain-openai python-dotenv


In [ ]:
import os
from fastapi import FastAPI, Header
from pydantic import BaseModel
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

app = FastAPI()


### Model agnostic gateway

In [ ]:
class ChatRequest(BaseModel):
    message: str

@app.post("/chat")
def chat_endpoint(req: ChatRequest, x_model: str = Header(default="openai:gpt-4o-mini")):
    """
    Gateway: wybór modelu po nagłówku X-Model.
    """
    if x_model.startswith("openai:"):
        model_name = x_model.split(":",1)[1]
        llm = ChatOpenAI(model=model_name, temperature=0)
    elif x_model == "oss:mock":
        # fallback: prosty model open-source (tutaj mock)
        return {"answer": f"(OSS fallback) Echo: {req.message}"}
    else:
        return {"error": f"Unknown model: {x_model}"}

    answer = llm.invoke(req.message)
    return {"answer": answer.content}


### Monitoring kosztów

In [ ]:
from datetime import datetime

costs = []

def log_cost(cost_usd: float):
    now = datetime.utcnow().isoformat()
    costs.append((now, cost_usd))
    total_today = sum(c for t,c in costs)  # uproszczony licznik
    if total_today > 50:
        print("⚠️ ALERT: Daily cost exceeded 50 USD!")
    return total_today

# Przykład logowania
log_cost(10)
log_cost(20)
log_cost(25)  # po tym powinien wysłać alert
